In [1]:
import os, glob
cwd = os.getcwd()
print(cwd)
new_dir = cwd + '/cnn/*.story'
files = glob.glob(new_dir)

for myfile in files:
    with open(myfile) as f:
        raw_docs = f.readlines()

/Users/tracyhuang/Documents/Berkeley/FinalProject


In [2]:
from nltk.tokenize import word_tokenize

tokenized_docs = [word_tokenize(doc) for doc in raw_docs]
#print tokenized_docs

In [3]:
import re
import string

regex = re.compile('[%s]' % re.escape(string.punctuation)) 

tokenized_docs_wo_punctuation = []

for review in tokenized_docs:
    
    new_review = []
    for token in review: 
        new_token = regex.sub(u'', token)
        if not new_token == u'':
            new_review.append(new_token)
    
    tokenized_docs_wo_punctuation.append(new_review)
    
#print tokenized_docs_no_punctuation

In [4]:
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

tokenized_docs_wo_stopwords = []
for doc in tokenized_docs_wo_punctuation:
    new_term_vector = []
    for word in doc:
        if not word in stopwords.words('english'):
            new_term_vector.append(word)
    tokenized_docs_wo_stopwords.append(new_term_vector)
            
#print tokenized_docs_no_stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/tracyhuang/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer

porter = PorterStemmer()
snowball = SnowballStemmer('english')
wordnet = WordNetLemmatizer()

preprocessed_docs = []

for doc in tokenized_docs_wo_stopwords:
    final_doc = []
    for word in doc:
        final_doc.append(porter.stem(word))
    preprocessed_docs.append(final_doc)

#print preprocessed_docs

In [7]:
from __future__ import absolute_import
from __future__ import division, print_function, unicode_literals

import math

from itertools import combinations
from collections import defaultdict
from collections import namedtuple
from operator import attrgetter

class TextSummarization():
    def __init__(self):
        if not callable(stemmer):
            raise ValueError("Stemmer needs to be callable")

    _stop_words = frozenset()

    @property
    def StopWords(self):
        return self._stop_words

    @StopWords.setter
    def StopWords(self, words):
        self._stop_words = frozenset(map(self.normalize_word, words))

    def call(self, document, sentences_count):
        ratings = self.rate_sentences(document)
        return self._get_best_sentences(document.sentences, sentences_count, ratings)
    
    def StemWord(self, word):
        return self._stemmer(self.normalize_word(word))

    def Normalization(self, word):
        return to_unicode(word).lower()

    def TopSentences(self, sentences, count, rating, *args, **kwargs):
        rate = rating
        if isinstance(rating, dict):
            assert not args and not kwargs
            rate = lambda s: rating[s]

        infos = (SentenceInfo(s, o, rate(s, *args, **kwargs))
            for o, s in enumerate(sentences))

        infos = sorted(infos, key=attrgetter("rating"), reverse=True)
        if not isinstance(count, ItemsCount):
            count = ItemsCount(count)
        infos = count(infos)
        infos = sorted(infos, key=attrgetter("order"))

        return tuple(i.sentence for i in infos)

    def RateSentences(self, document):
        sentences_words = [(s, self._to_words_set(s)) for s in document.sentences]
        ratings = defaultdict(float)

        for (sentence1, words1), (sentence2, words2) in combinations(sentences_words, 2):
            rank = self._rate_sentences_edge(words1, words2)
            ratings[sentence1] += rank
            ratings[sentence2] += rank

        return ratings

    def WordSet(self, sentence):
        words = map(self.normalize_word, sentence.words)
        return [self.stem_word(w) for w in words if w not in self._stop_words]

    def RateSentencesEdge(self, words1, words2):
        rank = 0
        for w1 in words1:
            for w2 in words2:
                rank += int(w1 == w2)

        if rank == 0:
            return 0.0

        assert len(words1) > 0 and len(words2) > 0
        norm = math.log(len(words1)) + math.log(len(words2))
        return 0.0 if norm == 0.0 else rank / norm